# Support vector regression 

## Imports

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, integrate
import seaborn as sns

import pprint
p = pprint.PrettyPrinter(indent=4)

import config as config

%matplotlib inline


## utilities and dataset
+ Imports untilities and generates the dataset as pandas Dataframes 
+ Check config.py for the training/testing set ratio (currently TRAIN_FRAC = 0.75) 

In [3]:
%run utilities.py
util = Utilities({}, use_json=False)
training_list, testing_list = util.get_datasets()

# turning the datasets into pandas DataFrames 
training_DF, testing_DF = util.create_dataframes(training_list, testing_list)

# generate the energy and dance measures 
training_DF, testing_DF = util.generate_energy_measure(training_DF, testing_DF)
training_DF, testing_DF = util.generate_dance_measure(training_DF, testing_DF)


## Making svr learners

In [4]:
from sklearn.svm import SVR
from sklearn import metrics 
# Look at utilities.py for RunAndTestLinearRegModel() 

In [5]:
%run utilities.py
def printerWrapper(input_string_list, train=False, test=False):
    
    for item in input_string_list:
        X_cols = item
        results = util.RunAndTestSVRModel(X_cols, training_DF, testing_DF)
        #  [x][1] = Mean squared error regression loss (bc its the smallest value lol)
        mserr_training = results[0]
        mserr_testing = results[1]
        if train: 
#             print "{:<40}".format(item), "training_error_rbf:\t", mserr_training[0]
            print "{:<40}".format(item), "training_error_lin:\t", mserr_training[1]
#             print "{:<40}".format(item), "training_error_poly:\t", mserr_training[2]
        if test:
#             print "{:<40}".format(item), "testing_error_rbf:\t\t\t", mserr_testing[0]
            print "{:<40}".format(item), "testing_error_lin:\t\t\t", mserr_testing[1]
#             print "{:<40}".format(item), "testing_error_poly:\t\t\t", mserr_testing[2]
        # print "The min error values were:", min(results[1]), min(results[2])
        # print "std of hot is:", results[3]
        # -------------------------------------------------


### Using metadata (art_hot and art_fam)

In [6]:
metadata = ['artist_familiarity', 'artist_hotttnesss', 'artist_hotttnesss artist_familiarity']
printerWrapper(metadata, False, True)

artist_familiarity                       testing_error_lin:			0.020997354516764582
artist_hotttnesss                        testing_error_lin:			0.02066705198772561
artist_hotttnesss artist_familiarity     testing_error_lin:			0.020068291575617073


### using acoustic features

In [7]:
allsegs = 'segments_avg tatums_avg beats_avg bars_avg sections_avg '
allrawacous = 'key loudness duration tempo time_signature'

raw_acous = []
for each in allsegs.split():
    raw_acous.append(each)

raw_acous.append(allsegs)


for each in allrawacous.split():
    raw_acous.append(each)   
raw_acous.append(allrawacous)
raw_acous.append((allrawacous+ ' ' + allsegs))

printerWrapper(raw_acous, True, True)


segments_avg                             training_error_lin:	0.02862152435177071
segments_avg                             testing_error_lin:			0.02731067392587206
tatums_avg                               training_error_lin:	0.028321506566793306
tatums_avg                               testing_error_lin:			0.02723018865928935
beats_avg                                training_error_lin:	0.028406245598129938
beats_avg                                testing_error_lin:			0.02728009713193656
bars_avg                                 training_error_lin:	0.02862146917708699
bars_avg                                 testing_error_lin:			0.02726403847465305
sections_avg                             training_error_lin:	0.028617720584006495
sections_avg                             testing_error_lin:			0.02729584722637941
segments_avg tatums_avg beats_avg bars_avg sections_avg  training_error_lin:	0.027974482790801586
segments_avg tatums_avg beats_avg bars_avg sections_avg  testing_error_lin:			0.0272

KeyboardInterrupt: 

### Find best energy measure

In [ ]:
energylsit = ['energy1', 'energy2', 'energy3', 'energy4', 'energy1 energy2 energy3 energy4']
printerWrapper(energylsit, True, True)



### dance now

In [ ]:
dancelist = ['dance1', 'dance2', 'dance3', 'dance4', 'dance1 dance2 dance3 dance4']

printerWrapper(dancelist, True, True)

### All Acoustic

In [ ]:
dancelist = 'dance1 dance2 dance3 dance4 '
energylist = 'energy1 energy2 energy3 energy4 '

acousitc =[]
acousitc.append(dancelist)
acousitc.append(energylist)
acousitc.append(dancelist+energylist)
acousitc.append("artist_familiarity artist_hotttnesss")
acousitc.append(dancelist+energylist+"artist_familiarity artist_hotttnesss")

printerWrapper(acousitc, True, True)


### Final 

In [ ]:
finalset = [dancelist+energylist+"artist_familiarity artist_hotttnesss"]

printerWrapper(finalset, True, True)


In [ ]:
rawacoustic = " key loudness duration tempo time_signature segments_avg tatums_avg beats_avg bars_avg sections_avg  "
above = ' dance1 dance2 dance3 dance4 energy1 energy2 energy3 energy4 artist_familiarity artist_hotttnesss  '
actual_all = rawacoustic+above

printerWrapper([actual_all], True, True)